## Cleaning for the base case: 
#### 1. check for NAs: don't drop, intropolate 
#### 2. check for unreasonable numbers
#### 3. calculate additional columns: daily bid-ask spread; n-day moving average of price, volume, vol
#### 4. calculate technical indicators: MACD, MA, RSI, etc.
#### 5. normalize all the variables

## Preparation for advanced models:
#### 1. combine with other datasets: crude oil, gold, UST, CAPE, vix
#### 2. check for unreasonable numbers
#### 3. normalize all the variables


In [8]:
import pandas as pd
ivv = pd.read_csv("ivv_daily.csv").drop(columns = ['NUMTRD','PERMNO','BID','ASK'])
#change column names
ivv.rename(columns={'BIDLO': 'low', 'ASKHI': 'high','PRC': 'close','OPENPRC': 'open','VOL':'volume'}, inplace=True)

In [9]:
#calculate additional columns

#1. close-open; (close-open)/close
ivv['close-open'] = ivv['close']-ivv['open']
ivv['close-open_%of_close'] = ivv['close-open']/ivv['close']

#2. high-low; (high-low)/close
ivv['high-low'] = ivv['high'] - ivv['low']
ivv['high-low_%of_close'] = ivv['high-low']/ivv['close']

In [10]:
#calculate the MACD, RSI, etc. 
#MACD
ema26 = ivv['close'].ewm(span=26).mean()
ema12 = ivv['close'].ewm(span=12).mean()
ivv['macd'] = ema12 -ema26

#RSI
delta = ivv['close'].diff()
delta = delta[1:]
up = delta.copy()
down = delta.copy()
up[up < 0] = 0
down[down > 0] = 0

roll_up = up.ewm(span=14).mean()
roll_down = down.ewm(span=14).mean()
rs = roll_up / roll_down
rsi = 100 - (100/(1+rs))

ivv['rsi'] = rsi

#50-, 100-, 200- moving average
ivv['50d_avg'] = ivv['close'].rolling(50).mean()
ivv['100d_avg'] = ivv['close'].rolling(100).mean()
ivv['200d_avg'] = ivv['close'].rolling(200).mean()

#stochastic oscillator
low_14 = ivv['close'].rolling(14).min()
high_14 = ivv['close'].rolling(14).max()
ivv['so'] = 100*(ivv['close'] - low_14)/(high_14 - low_14)


In [11]:
ivv.to_csv('ivv_raw.csv')

In [12]:
ivv_normalized=(ivv-ivv.mean())/ivv.std()
ivv_normalized['date'] = ivv['date']
ivv_normalized['close'] = ivv['close']

ivv_normalized = ivv_normalized[200:]
#ivv_normalized.to_csv("ivv_normal.csv")
ivv_normalized.head()
len(ivv_normalized)

4357

In [14]:
ivv_normalized.head()

,date,low,high,close,volume,open,close-open,close-open_%of_close,high-low,high-low_%of_close,macd,rsi,50d_avg,100d_avg,200d_avg,so
200,20010307,-0.439921,-0.455396,126.55,-0.669027,-0.438876,-0.124481,-0.119906,-0.293656,-0.254819,-1.713257,-0.062340,-0.328303,-0.265805,-0.107318,-0.765715
201,20010308,-0.429719,-0.447085,126.92,-1.015534,-0.439084,0.174580,0.180720,-0.327246,-0.289116,-1.572642,0.318968,-0.330083,-0.267201,-0.108762,-0.303249
202,20010309,-0.494888,-0.480540,123.47,-0.244510,-0.468421,-1.430906,-1.471366,0.240059,0.297527,-1.636192,-0.013111,-0.333621,-0.270195,-0.110353,-1.609645
203,20010312,-0.609195,-0.550982,117.78,-0.285677,-0.534586,-3.406283,-3.679961,1.023836,1.177954,-1.982363,-0.009977,-0.340270,-0.274561,-0.112813,-1.609645
204,20010313,-0.615233,-0.596905,119.72,-1.009871,-0.601999,0.670392,0.718906,0.307240,0.396222,-2.125731,-0.012221,-0.346038,-0.277851,-0.114912,-1.036723


### Merge the ivv dataset with gold price, oil price, ust, vix

In [15]:
oil = pd.read_csv("oil.csv")
gold = pd.read_csv("gold.csv")
vix = pd.read_csv("vix_close.csv")
ust = pd.read_csv("ust.csv")

In [16]:
#normalize all the series
oil_normalized = (oil-oil.mean())/oil.std()
oil_normalized['date'] = pd.to_numeric(oil['date'])

gold_normalized = (gold-gold.mean())/gold.std()
gold_normalized['date'] = pd.to_numeric(gold['date'])

vix_normalized = (vix-vix.mean())/vix.std()
vix_normalized['date'] = pd.to_numeric(vix['date'])

ust_normalized = (ust-ust.mean())/ust.std()
ust_normalized['date'] = pd.to_numeric(ust['date'])

In [17]:
merged = pd.merge(ivv_normalized, oil_normalized, how='left', on=['date'])
merged = pd.merge(merged,gold_normalized, how = 'left', on = ['date'])
merged = pd.merge(merged,vix_normalized, how = 'left', on = ['date'])
merged = pd.merge(merged,ust_normalized, how = 'left', on = ['date'])

In [18]:
#interpolate the NAs, using pandas linear interpolation
#since there are only 250 missing data points (<0.3% of the total data points), use simple linear interpolation
merged.isnull().values.ravel().sum()

250

In [19]:
merged = merged.interpolate()
merged.isnull().values.ravel().sum()

0

In [20]:
merged[merged.isnull().any(axis=1)]

,date,low,high,close,volume,open,close-open,close-open_%of_close,high-low,high-low_%of_close,...,so,oil,gold,vix,ust1,ust2,ust5,ust10,ust5vs2,ust10vs2


In [21]:
merged.to_csv("merged.csv")

In [22]:
merged.head()

,date,low,high,close,volume,open,close-open,close-open_%of_close,high-low,high-low_%of_close,...,so,oil,gold,vix,ust1,ust2,ust5,ust10,ust5vs2,ust10vs2
0,20010307,-0.439921,-0.455396,126.55,-0.669027,-0.438876,-0.124481,-0.119906,-0.293656,-0.254819,...,-0.765715,-0.608423,-0.901128,0.621340,1.417591,1.358131,1.261813,1.200361,-1.015095,-0.991021
1,20010308,-0.429719,-0.447085,126.92,-1.015534,-0.439084,0.174580,0.180720,-0.327246,-0.289116,...,-0.303249,-0.624854,-0.892326,0.643071,1.423382,1.372399,1.268407,1.214223,-1.045676,-0.999702
2,20010309,-0.494888,-0.480540,123.47,-0.244510,-0.468421,-1.430906,-1.471366,0.240059,0.297527,...,-1.609645,-0.635309,-0.879480,0.813084,1.455180,1.385555,1.289462,1.239362,-1.029111,-0.990609
3,20010312,-0.609195,-0.550982,117.78,-0.285677,-0.534586,-3.406283,-3.679961,1.023836,1.177954,...,-1.609645,-0.648155,-0.877577,1.413883,1.416330,1.350912,1.259006,1.202382,-0.998047,-0.974132
4,20010313,-0.615233,-0.596905,119.72,-1.009871,-0.601999,0.670392,0.718906,0.307240,0.396222,...,-1.036723,-0.666676,-0.888996,1.059795,1.413452,1.369915,1.274295,1.232737,-1.019323,-0.969239


### Check the standard deviation of each series, to decide which one probabily can be left out for the CNN model

In [24]:
merged.std()

date                    50060.792260
low                         1.021570
high                        1.021678
close                      49.107338
volume                      0.999575
open                        1.021636
close-open                  0.988563
close-open_%of_close        0.993536
high-low                    1.019471
high-low_%of_close          1.019843
macd                        1.003617
rsi                         0.299232
50d_avg                     1.016643
100d_avg                    1.011311
200d_avg                    1.000113
so                          0.996776
oil                         0.913153
gold                        1.074721
vix                         1.133866
ust1                        0.837241
ust2                        0.842708
ust5                        0.863660
ust10                       0.896431
ust5vs2                     0.940923
ust10vs2                    0.916028
dtype: float64